In [ ]:
# Reference used to understand Retinanet and run the code: 
# https://medium.com/@van.evanfebrianto/how-to-train-custom-object-detection-models-using-retinanet-aeed72f5d701#64db

# Code to ensure that colab does not disconnect
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)



In [ ]:
from os import listdir
from os.path import isfile, join
from google.colab import drive
from keras_retinanet import models
drive.mount('/content/drive')

In [ ]:
%%capture
!python setup.py build_ext --inplace
!pip install keras_resnet
!pip install .

In [ ]:
%cd /content/drive/MyDrive/Retinanet/keras-retinanet-main/

In [ ]:
# Create train test files
# Reference for creating train, test, validation files
# https://medium.com/@van.evanfebrianto/how-to-train-custom-object-detection-models-using-retinanet-aeed72f5d701#64db
data_dir = "dataset/JPEGImages/"
files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f))]

i = 0
with open("val.txt", "w") as val:
    for f in files:
        i += 1
        if (i % 10) == 0:
            if f.split("/")[-1].split(".")[-1] == 'jpg':
              name = ".".join(f.split("/")[-1].split(".")[:-1])
            else:
              name = f.split("/")[-1]
            if '.DS_Store' in name:
              continue
            val.write(name + "\n")       

i = 0
with open("test.txt", "w") as test:
    for f in files:
        i += 1
        if (i % 10) == 1:
            if f.split("/")[-1].split(".")[-1] == 'jpg':
              name = ".".join(f.split("/")[-1].split(".")[:-1])
            else:
              name = f.split("/")[-1]
            if '.DS_Store' in name:
              continue
            test.write(name + "\n")
            

i = 0
with open("train.txt", "w") as train:
    for f in files:
        i += 1
        if (i % 10) != 0:
            if f.split("/")[-1].split(".")[-1] == 'jpg':
              name = ".".join(f.split("/")[-1].split(".")[:-1])
            else:
              name = f.split("/")[-1]
            if '.DS_Store' in name:
              continue
            train.write(name + "\n")

# print(i)
val.close()
test.close()
train.close()

In [ ]:
# Train code for epochs = 10, batch size = 1, steps = 500, lr = 0.00001, backbone = ResNet50
!python keras_retinanet/bin/train.py --freeze-backbone \
--lr 0.00001 \
--random-transform \
--multi-gpu 1\
--gpu 0\
--weights snapshots/resnet50_coco_best_v2.1.0.h5 \
--epoch 10 \
--batch-size 1 \
--steps 500 \
--snapshot-path snapshots \
--tensorboard-dir tf-logs \
pascal ./dataset

In [ ]:
# Train code for epochs 20, batch size = 1, steps = 500, lr = 0.00001, backbone = ResNet50
!python keras_retinanet/bin/train.py --freeze-backbone \
--lr 0.00001 \
--random-transform \
--multi-gpu 1\
--gpu 0\
--weights snapshots/resnet50_coco_best_v2.1.0.h5 \
--epoch 20 \
--batch-size 1 \
--steps 500 \
--snapshot-path snapshots \
--tensorboard-dir tf-logs \
pascal ./dataset

In [ ]:
# Convert training model to inference model
!python keras_retinanet/bin/convert_model.py snapshots/resnet50_pascal_10.h5 weights/resnet50_pascal_10.h5

In [ ]:
# Convert training model to inference model
!python keras_retinanet/bin/convert_model.py snapshots/resnet50_pascal_20.h5 weights/resnet50_pascal_20.h5

In [ ]:
# Evaluating the performance of model with 10 epochs on test dataset 
!python keras_retinanet/bin/evaluate.py pascal dataset \
weights/resnet50_pascal_10.h5

In [ ]:
# Evaluating the performance of model with 20 epochs on test dataset 
!python keras_retinanet/bin/evaluate.py pascal dataset \
weights/resnet50_pascal_20.h5